# Sobre este Notebook

Foi necessário converter o arquivo de pontos para o formato GeoJSON e KML.
Vou te passar um exemplo de solicitação e conclusão e, a partir este material, você pode até criar um video, ou mesmo fazer uma postagem no medium ou artigo no linkedin.

## Os objetivos iniciais são:

1.   Montar o Mapa com os poligonos da região
2.   No mesmo mapa, incluir todos os pontos de oorrência
3.   Processar o DataSet de Ocorrência, demonstrando quem está ou não dentro dos raios
4.   Fazer novo Mapa, com as regiões e também com as ocorrências

## Conclusões ou ideias de próximos trabalhos quanto a esta análise:

* Sendo analista, devemos tirar algumas conclusões, mesmo básicas, informando quantos eventos ocorreram dentro e fora dos círculos.
* Quais seriam nossos eventos de interesse, os que ocorrem dentro ou fora dos políogonos?
* Que dados podem ser levantados e entregues ao pessoal do Negócio, com o objetivo de mitigar a ocorrência de eventos ou "punição" de clientes que não cumprem as regras?








# Fontes de estudo para este Notebook

* https://egmara-santos.medium.com/utilizando-python-para-verificar-coordenadas-geogr%C3%A1ficas-em-um-mapa-kml-c4d61541e5ce
* https://towardsdatascience.com/creating-a-simple-map-with-folium-and-python-4c083abfff94


# Instalação de bibliotecas no ambiente do Google Collab

In [ ]:
!python3 -m pip install geopandas
!python3 -m pip install shapely

# Importação de Bibliotecas e Montagem do drive de Dados

In [3]:
import json
import os
import numpy as np
import glob
from folium import GeoJson
import geopandas as gpd
from shapely.geometry import Polygon, mapping
import fiona
fiona.drvsupport.supported_drivers['KML'] = 'rw'
from fiona.drvsupport import supported_drivers
from shapely.geometry import Point, Polygon
import pandas as pd
from folium import plugins
import folium

supported_drivers['LIBKML'] = 'rw'

from google.colab import drive

drive.mount('/content/drive')

WORK_DIR = '/content/drive/MyDrive/Colab Notebooks/testegeopandas'


Mounted at /content/drive


# Definição das variáveis com os nomes dos arquivos da Base

In [4]:
arquivo_json_raio_200km_json = os.path.join(WORK_DIR, 'base')
arquivo_json_raio_200km_json = os.path.join(arquivo_json_raio_200km_json, 'Raio_de_200km_PGR.geojson')

arquivo_json_raio_200km_kml = os.path.join(WORK_DIR, 'base')
arquivo_json_raio_200km_kml = os.path.join(arquivo_json_raio_200km_kml, 'Raio_de_200km_PGR.kml')

arquivo_json_roubo_furto = os.path.join(WORK_DIR, 'base')
arquivo_json_roubo_furto = os.path.join(arquivo_json_roubo_furto, 'Roubo_Furto.geojson')


# Caso o arquivo for KMZ, converter usando o serviço https://mygeodata.cloud/converter/kmz-to-kml

def get_ultimo_arquivo_kml():
  diretorio_base_cliente = os.path.join(WORK_DIR, 'base')
  diretorio_base_cliente = os.path.join(diretorio_base_cliente, '*.kml')

  return  max(glob.glob(diretorio_base_cliente), key=os.path.getmtime)

# print("Encontrado o arquivo:" , get_ultimo_arquivo_kml())


# Leitura da Base de Dados KML

In [ ]:
df_regioes = gpd.read_file(arquivo_json_raio_200km_json)
df_ocorrencias = gpd.read_file(arquivo_json_roubo_furto)
mapa_kml = gpd.read_file(arquivo_json_raio_200km_kml, driver='KML')

# Conversão do campo de conjunto de pontos de LineString para Polygon

In [ ]:
# Baseado no link : https://stackoverflow.com/questions/2964751/how-to-convert-a-geos-multilinestring-to-polygon
def retorna_poligono(linha):
  geom = linha["geometry"]
  all_coords = mapping(geom)['coordinates']
  lats = [x[1] for x in all_coords]
  lons = [x[0] for x in all_coords]
  return Polygon(zip(lons, lats))


df_regioes["poligonos"] = df_regioes.apply(retorna_poligono, axis=1)
#df_regioes.head()

mapa_kml["poligonos"] = mapa_kml.apply(retorna_poligono, axis=1)
#mapa_kml.head()


# Geração do Mapa

In [ ]:
fig = folium.Map(location=[-14.2400732, -53.1805017], # Peguei no site: https://latitude.to/map/br/brazil
                 width=600,
                 height=800,
                 tiles="Cartodb Positron",
                zoom_start=4, # Achei um Zoom que se encaixava
                zoom_control=True)

geo_json_mapa = json.load(open(arquivo_json_raio_200km_json))
folium.Choropleth(
    geo_data = geo_json_mapa,
    fill_color = "red",
    fill_opacity = 0.5,
    line_color = "red",
    line_opacity = 0.5
).add_to(fig);

minimap = plugins.MiniMap()


# Carregamento dos eventos de Furto e Roubo

In [ ]:
#
file = open(arquivo_json_roubo_furto, encoding="utf8")
text = file.read()

GeoJson(text,).add_to(fig, );


# Renderizaçao do Mapa

In [ ]:
fig.add_child(minimap)

# Verificar quais eventos pertencem ou não ao Raio

In [ ]:
#EM DESENVOLVIMENTO


# Função que trata linha a linha
def funcao_verificacao(linha):
  check = mapa_kml.poligonos.contains(linha["geometry"]).values

  # print(check, np.any(check))

  return np.any(check)

df_ocorrencias["pertence_ao_raio"] = df_ocorrencias.apply(funcao_verificacao, axis=1)

df_nao_pertence_ao_raio = df_ocorrencias[df_ocorrencias["pertence_ao_raio"]==0]

In [ ]:
print(f'Existem {df_nao_pertence_ao_raio.shape[0]} eventos que ocorreram fora do raio')

Existem 6 eventos que ocorreram fora do raio


Eu acho que a base de locais que ocorreram eventos fora do Raio, é passível de punição ao cliente, e, se exibir do problema

E, o evento de interesse, é analisar os problemas, que ocorre dentro dos raios.

Exemplo, temos que criar uma Analise Base Table (ABT).. com campos de horário.. talvez ligando com data de cadastro do cliente, etc.. e, tentar usar algoritimos de Inteligência Artifical como o objetivo de prever quando ocorre acidente, dias, perfil da empresa e da carga, etc.



# Mapa com Eventos fora do Raios

In [ ]:
# Carregamento do Mapa e Regiões
fig = folium.Map(location=[-14.2400732, -53.1805017], # Peguei no site: https://latitude.to/map/br/brazil
                 width=800,
                 height=1000,
                tiles="Cartodb Positron",
                fillColor='#DEFFDE',
                color='#AECCAE',
                  zoom_start=4, # Achei um Zoom que se encaixava
                 zoom_control=True)

geo_json_mapa = json.load(open(arquivo_json_raio_200km_json))
folium.Choropleth(
    geo_data = geo_json_mapa,
    fill_color = "red",
    fill_opacity = 0.5,
    line_color = "red",
    line_opacity = 0.5
).add_to(fig);

minimap = plugins.MiniMap()

# Atribuir coorenadas e eventos fora do mapa
def carregar_pontos_geometricos(linha):
  texto_popup = 'Tipo: '+linha["TIPO"]+'\n'+\
                'Cidade: '+linha["CIDADE_DO_EVENTO"]+'/'+linha["UF"]


  folium.Marker([linha["Latitude"], linha["Longitude"]], popup=texto_popup,
                tooltip=linha["TIPO"]+' em '+linha["LOCAL_DO_EVENTO"]+", "+linha["DATA_EVENTO"],
                icon=folium.Icon(color="gray", icon="info-sign")).add_to(fig)

df_nao_pertence_ao_raio.apply(carregar_pontos_geometricos, axis=1)

fig


In [ ]:
df_nao_pertence_ao_raio.head(10)

,Name,PROCESSO,DATA_EVENTO,TIPO,LOCAL_DO_EVENTO,SIGLA,Latitude,Longitude,CIDADE_DO_EVENTO,UF,CEP,geometry,pertence_ao_raio
18,ROUBO,428872,7/18/2021,ROUBO,Rua Dona Alzira,PU,-30.0006,-51.1397,Porto Alegre,RS,91110-010,POINT (-51.13968 -30.00060),False
23,ROUBO,430041,8/17/2021,ROUBO,Rua Dona Alzira,-,-30.0072,-51.1381,Porto Alegre,RS,91110-010,POINT (-51.13810 -30.00720),False
24,ROUBO,430808,9/1/2021,ROUBO,Dependeciais LATAM,PU,-8.7408,-63.8962,Porto Velho,RO,NaN,POINT (-63.89620 -8.74080),False
30,ROUBO,432974,10/28/2021,ROUBO,Dona alzira,RS,-30.0072,-51.1381,Butia,RS,91110-010,POINT (-51.95579 -30.12368),False
40,ROUBO,437138,1/14/2022,ROUBO,R William Booth,-,-25.5019,-49.2262,Curitiba,PR,81730-080,POINT (-49.22616 -25.50193),False
73,ROUBO,436449,12/30/2021,ROUBO,R J da Penha,-,-37.3170,-38.5185,Fortaleza,CE,NaN,POINT (-38.61597 -3.80290),False
